# Syndrome Decoding and Incomplete Decoding

**Mahan Madani - 99222092**

Note: this file contains the answer to the both the Syndrome Decoding and Incomplete decoding projects.


## Table of Contents:

- [Preparations and Utility Functions](##-Preparations-and-Utility-Functions)
- [Linear Block Code Class](##Linear-Block-Code-Class)
- [Examples](##Examples)
    - [Example 1](###Example-1)
    - [Example 2](###Example-2)
    - [Example 3](###Example-3)


## Preparations and Utility Functions

In [472]:
import numpy as np
import sympy as sp
import pprint

The Functions defined in this section are all used in the constructor of the Linear Block Code class. The block code is defined using a list of linearly independent vectors. All attributes of the code including the Generator and Parity matrices are set up using the initial vectors.

* Note that for this project, I assumed all codes are binary and are craeted over GF(2)

In [473]:
# Build a standardized generator matrix from in the form [I | A]

def build_generator_matrix(vectors):
    matrix = sp.Matrix(vectors)    
    echelon_matrix, _ = matrix.rref(simplify=True)

    echelon_matrix = [row for row in echelon_matrix.tolist() if any(row)]  # remove rows filled with 0
    
    return np.array(echelon_matrix, dtype=int) % 2

In [474]:
# Build a standardized parity matrix in the form [B | I]

def build_parity_matrix(generator_matrix):
    k, n = generator_matrix.shape
    redundancy_matrix = generator_matrix[:, k:]
    
    return np.concatenate([redundancy_matrix.T, np.identity(n-k, dtype=int)], axis=1)

The minimum hamming distance of a code is equal to its minimum weight, so by using the `np.count_nonzero()` function we can calculate the minimum hamming distance by iterating over all of the code words and finding the smallest weight.

The `minimum_hamming_dist` calculates the minimum Hamming distance of a linear block code

In [475]:
def minimum_hamming_dist(linear_code):
    min_dist = linear_code.n
    
    for vector in linear_code.generate_codewords():
        hamming_dist = np.count_nonzero(vector)
        min_dist = hamming_dist if (hamming_dist < min_dist and hamming_dist > 0) else min_dist
    
    return min_dist

The `generate_coset_table()` function iterates over all possible binary values with maximum length n. They will then be partitioned using into cosets. `coset_table` maps each syndrome to its respective coset (which is the collection of all binary values with that specific syndrome value)


Note that `coset_table` is a one-to-many mapping, meaning that it maps a syndrome to all members of a coset.

In [476]:
def generate_coset_table(linear_code):
    coset_table = dict()
    
    # generate all binary messages from 0 to 2^n
    for i in range(2**linear_code.n):
        codeword = np.array(list(map(int, bin(i)[2:].zfill(linear_code.n)))).tolist()
        syndrome = linear_code.compute_syndrome(codeword)
        
        if syndrome in coset_table:
            coset_table[syndrome].append(codeword)
        else:
            coset_table[syndrome] = [codeword]
    
    return coset_table

The `generate_syndrome_table()` function creates a dictionary mapping each syndrome to the coset leader from the `coset_table`.

Unlike `coset_table`, the `syndrome_table` dictionary uses one-to-one mapping, meaning that each syndrome is mapped to exactly one coset leader. The coset leader is chosen from the `coset_table`, the coset with the smallest weight is chosen as the leader.

In [477]:
def generate_syndrome_table(coset_table):
    coset_leader_table = dict()

    for syndrome in coset_table:
        coset_leader = coset_table[syndrome][0]

        for codeword in coset_table[syndrome]:
            coset_leader = codeword if np.count_nonzero(codeword) < np.count_nonzero(coset_leader) else coset_leader

        coset_leader_table[syndrome] = coset_leader

    return coset_leader_table

## Linear Block Code Class

This class is used to create linear block codes from a list of vectors.

The constructor call many of the functions defined in the previous section to set various values and find the coset_table and the coset_leader_table.

**Syndrom Decoding**: Codes made using this class can utilize syndrome values to detect errors and correct them. The syndrome table is generated using the coset table.

**Incomplete Decoding**: Even if a code cannot be corrected, errors can still be detected in certain cases.

In [478]:
class LinearCode:
    
    def __init__(self, vectors):
        self.G = build_generator_matrix(vectors)
        self.H = build_parity_matrix(self.G)

        self.k, self.n = self.G.shape
        self.d = minimum_hamming_dist(self)
        self.correction_thershold = (self.d - 1)//2     # The maximum number of error that can be corrected

        self.coset_table = generate_coset_table(self)
        self.syndrome_table = generate_syndrome_table(self.coset_table)  # Maps syndromes to coset leaders

    
    def generate_codewords(self):   # Generate all possible codewords from the generator matrix
        codewords = []
        for i in range(2**self.k):
            message = np.array(list(map(int, bin(i)[2:].zfill(self.k))))  # generate all binary messages from 0 to 2^k
            codeword = np.dot(message, self.G) % 2                        # find the encoded value of each binary message
            codewords.append(codeword)
            
        return np.array(codewords)
    
    
    def compute_syndrome(self, r) -> str:   # Compute the syndrome value of a received message. the value is stored as a string for later usage.
        if len(r) == self.n:
            syndrome = np.dot(r, self.H.T) % 2
            return np.array2string(syndrome, separator='')[1:-1]
        else:
            return None
        
    
    def decode_message(self, r):
        if len(r) == self.n:
            syndrome = self.compute_syndrome(r)

            if syndrome != '0'*len(syndrome):
                error_vector = np.array(self.syndrome_table[syndrome])     # The Error vector is the coset leader linked to the syndrome 

                print('Error Found!')
                print(f'Syndrome: {syndrome}')
                print(f'Error Vector: {error_vector}')

                if np.count_nonzero(error_vector) > self.correction_thershold:
                    print(f'Error correction not possible. Only {self.correction_thershold} error(s) can be corrected.')
                    return
                else:
                    r = (r + self.syndrome_table[syndrome]) % 2    # The error can be corrected by adding the error vector to the message
                    print(f'Corrected codeword: {r}')
            else:
                print('No errors detected.')
            
            print(f"Decoded message: {r[:self.k]}")

        else:
            print('Error detected: Wrong message length')
    

    def encode_message(self, message):
        if len(message) == self.k:
            return np.dot(message, self.G)
        else:
            return None

## Examples

### Example 1

Repetition code-(5,2)

In [479]:
vectors1 = [[1,1,1,1,1], [0,0,0,0,0]]
linear_code_1 = LinearCode(vectors1)

In [480]:
print("Generator Matrix")
print(linear_code_1.G)

print("\nParity Check Matrix")
print(linear_code_1.H)

print("\nMinimum Hamming Distance")
print(linear_code_1.d)

print("\nH * G^T")
print(np.matmul(linear_code_1.H, linear_code_1.G.T) % 2)

Generator Matrix
[[1 1 1 1 1]]

Parity Check Matrix
[[1 1 0 0 0]
 [1 0 1 0 0]
 [1 0 0 1 0]
 [1 0 0 0 1]]

Minimum Hamming Distance
5

H * G^T
[[0]
 [0]
 [0]
 [0]]


In [481]:
print("Coset Table")
pprint.pprint(linear_code_1.coset_table)

Coset Table
{'0000': [[0, 0, 0, 0, 0], [1, 1, 1, 1, 1]],
 '0001': [[0, 0, 0, 0, 1], [1, 1, 1, 1, 0]],
 '0010': [[0, 0, 0, 1, 0], [1, 1, 1, 0, 1]],
 '0011': [[0, 0, 0, 1, 1], [1, 1, 1, 0, 0]],
 '0100': [[0, 0, 1, 0, 0], [1, 1, 0, 1, 1]],
 '0101': [[0, 0, 1, 0, 1], [1, 1, 0, 1, 0]],
 '0110': [[0, 0, 1, 1, 0], [1, 1, 0, 0, 1]],
 '0111': [[0, 0, 1, 1, 1], [1, 1, 0, 0, 0]],
 '1000': [[0, 1, 0, 0, 0], [1, 0, 1, 1, 1]],
 '1001': [[0, 1, 0, 0, 1], [1, 0, 1, 1, 0]],
 '1010': [[0, 1, 0, 1, 0], [1, 0, 1, 0, 1]],
 '1011': [[0, 1, 0, 1, 1], [1, 0, 1, 0, 0]],
 '1100': [[0, 1, 1, 0, 0], [1, 0, 0, 1, 1]],
 '1101': [[0, 1, 1, 0, 1], [1, 0, 0, 1, 0]],
 '1110': [[0, 1, 1, 1, 0], [1, 0, 0, 0, 1]],
 '1111': [[0, 1, 1, 1, 1], [1, 0, 0, 0, 0]]}


In [482]:
print('Syndrome Table (Syndromes mapped to Coset Leaders)')
pprint.pprint(linear_code_1.syndrome_table)

Syndrome Table (Syndromes mapped to Coset Leaders)
{'0000': [0, 0, 0, 0, 0],
 '0001': [0, 0, 0, 0, 1],
 '0010': [0, 0, 0, 1, 0],
 '0011': [0, 0, 0, 1, 1],
 '0100': [0, 0, 1, 0, 0],
 '0101': [0, 0, 1, 0, 1],
 '0110': [0, 0, 1, 1, 0],
 '0111': [1, 1, 0, 0, 0],
 '1000': [0, 1, 0, 0, 0],
 '1001': [0, 1, 0, 0, 1],
 '1010': [0, 1, 0, 1, 0],
 '1011': [1, 0, 1, 0, 0],
 '1100': [0, 1, 1, 0, 0],
 '1101': [1, 0, 0, 1, 0],
 '1110': [1, 0, 0, 0, 1],
 '1111': [1, 0, 0, 0, 0]}


In [483]:
print("Valid Codewords")
print(linear_code_1.generate_codewords())

Valid Codewords
[[0 0 0 0 0]
 [1 1 1 1 1]]


In [484]:
r = np.array([0,1,0,0,1])
linear_code_1.decode_message(r)

Error Found!
Syndrome: 1001
Error Vector: [0 1 0 0 1]
Corrected codeword: [0 0 0 0 0]
Decoded message: [0]


In [485]:
r = np.array([1,1,1,1,1])
linear_code_1.decode_message(r)

No errors detected.
Decoded message: [1]


### Example 2

In [486]:
vectors2 = [[1,0,1,1,0], [0,1,1,1,1]]
linear_code_2 = LinearCode(vectors2)

In [487]:
print("Generator Matrix")
print(linear_code_2.G)

print("\nParity Check Matrix")
print(linear_code_2.H)

print("\nMinimum Hamming Distance")
print(linear_code_2.d)

print("\nH * G^T")
print(np.matmul(linear_code_2.H, linear_code_2.G.T) % 2)

Generator Matrix
[[1 0 1 1 0]
 [0 1 1 1 1]]

Parity Check Matrix
[[1 1 1 0 0]
 [1 1 0 1 0]
 [0 1 0 0 1]]

Minimum Hamming Distance
3

H * G^T
[[0 0]
 [0 0]
 [0 0]]


In [488]:
print("Coset Table")
pprint.pprint(linear_code_2.coset_table)

Coset Table
{'000': [[0, 0, 0, 0, 0], [0, 1, 1, 1, 1], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1]],
 '001': [[0, 0, 0, 0, 1], [0, 1, 1, 1, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 0]],
 '010': [[0, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [1, 1, 0, 1, 1]],
 '011': [[0, 0, 0, 1, 1], [0, 1, 1, 0, 0], [1, 0, 1, 0, 1], [1, 1, 0, 1, 0]],
 '100': [[0, 0, 1, 0, 0], [0, 1, 0, 1, 1], [1, 0, 0, 1, 0], [1, 1, 1, 0, 1]],
 '101': [[0, 0, 1, 0, 1], [0, 1, 0, 1, 0], [1, 0, 0, 1, 1], [1, 1, 1, 0, 0]],
 '110': [[0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [1, 0, 0, 0, 0], [1, 1, 1, 1, 1]],
 '111': [[0, 0, 1, 1, 1], [0, 1, 0, 0, 0], [1, 0, 0, 0, 1], [1, 1, 1, 1, 0]]}


In [489]:
print('Syndrome Table (Syndromes mapped to Coset Leaders)')
pprint.pprint(linear_code_2.syndrome_table)

Syndrome Table (Syndromes mapped to Coset Leaders)
{'000': [0, 0, 0, 0, 0],
 '001': [0, 0, 0, 0, 1],
 '010': [0, 0, 0, 1, 0],
 '011': [0, 0, 0, 1, 1],
 '100': [0, 0, 1, 0, 0],
 '101': [0, 0, 1, 0, 1],
 '110': [1, 0, 0, 0, 0],
 '111': [0, 1, 0, 0, 0]}


In [490]:
print("Valid Codewords")
print(linear_code_2.generate_codewords())

Valid Codewords
[[0 0 0 0 0]
 [0 1 1 1 1]
 [1 0 1 1 0]
 [1 1 0 0 1]]


In [491]:
r = np.array([0,1,0,0,1])
linear_code_2.decode_message(r)

Error Found!
Syndrome: 110
Error Vector: [1 0 0 0 0]
Corrected codeword: [1 1 0 0 1]
Decoded message: [1 1]


In [492]:
r = np.array([0,0,1,0,1])
linear_code_2.decode_message(r)

Error Found!
Syndrome: 101
Error Vector: [0 0 1 0 1]
Error correction not possible. Only 1 error(s) can be corrected.


In [493]:
r = np.array([0,1,1,1,1])
linear_code_2.decode_message(r)

No errors detected.
Decoded message: [0 1]


### Example 3

In [494]:
vectors3 = [
    [1,1,1,0,0,0,0],
    [1,0,0,1,1,0,0],
    [0,1,0,1,0,1,0],
    [1,1,0,1,0,0,1]
]

linear_code_3 = LinearCode(vectors3)

In [495]:
linear_code_3.syndrome_table

{'000': [0, 0, 0, 0, 0, 0, 0],
 '001': [0, 0, 0, 0, 0, 0, 1],
 '010': [0, 0, 0, 0, 0, 1, 0],
 '011': [1, 0, 0, 0, 0, 0, 0],
 '100': [0, 0, 0, 0, 1, 0, 0],
 '101': [0, 1, 0, 0, 0, 0, 0],
 '110': [0, 0, 1, 0, 0, 0, 0],
 '111': [0, 0, 0, 1, 0, 0, 0]}

In [496]:
linear_code_3.correction_thershold

1

In [497]:
linear_code_3.generate_codewords()

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 1],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 0, 0, 1],
       [0, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 1],
       [0, 1, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1]], dtype=int32)

In [498]:
r = np.array([1,0,1,1,1,0,1])
linear_code_3.decode_message(r)

Error Found!
Syndrome: 111
Error Vector: [0 0 0 1 0 0 0]
Corrected codeword: [1 0 1 0 1 0 1]
Decoded message: [1 0 1 0]
